In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
import numpy as np
import json
from equistore import Labels, TensorBlock, TensorMap
from utils.builder import TensorBuilder
import ase.io
from itertools import product
from utils.acdc_mini import acdc_standardize_keys, cg_increment, cg_combine, _remove_suffix
from utils.clebsh_gordan import ClebschGordanReal
from utils.hamiltonians import hamiltonian_features
import matplotlib.pyplot as plt

from utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from rascal.representations import SphericalExpansion
import copy
from utils.model_hamiltonian import *
from itertools import product

In [3]:
from utils.mp_utils import * 

In [171]:
frames1 = ase.io.read("data/hamiltonian/water-hamiltonian/water_coords_1000.xyz",":10")
frames2 = ase.io.read("data/hamiltonian/ethanol-hamiltonian/ethanol_4500.xyz",":1")
frames3= [ase.build.molecule('NH3')]
frames = frames1+frames2+frames3#+frames2
for f in frames:
    f.cell = [100,100,100]
    f.positions += 50

In [172]:
rascal_hypers = {
    "interaction_cutoff": 3.5,
    "cutoff_smooth_width": 0.3,
    "max_radial": 1,
    "max_angular": 2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
#     "expansion_by_species_method": "user defined",
#     "global_species": [1,6,8,7]
    
}

spex = RascalSphericalExpansion(rascal_hypers)
rhoi = spex.compute(frames)

In [173]:
cg = ClebschGordanReal(5)

In [174]:
pairs = RascalPairExpansion(rascal_hypers)
gij = pairs.compute(frames)

In [175]:
rho1i = acdc_standardize_keys(rhoi)
rho1i.keys_to_properties(['species_neighbor'])
gij =  acdc_standardize_keys(gij)

rho1ip = relabel_key_contract(rho1i)
gji = relabel_key_contract(gij)

In [176]:
rho1ip #.block(1).samples

TensorMap with 12 blocks
keys: ['order_nu' 'inversion_sigma' 'spherical_harmonics_l' 'species_contract']
           1             1                    0                    1
           1             1                    0                    6
           1             1                    0                    7
        ...
           1             1                    2                    6
           1             1                    2                    7
           1             1                    2                    8

In [177]:
rho2i = cg_increment(rho1i, rho1i, lcut=1, other_keys_match=["species_center"], clebsch_gordan=cg)
# rho2ip = relabel_key_contract(rho2i)

In [178]:
rho3i =  cg_increment(rho2i, rho1i, lcut=1, other_keys_match=["species_center"], clebsch_gordan=cg)

In [179]:
%%time
rho1ij = cg_increment(rho1i, gij, lcut=2, other_keys_match=["species_center"], clebsch_gordan=cg)

CPU times: user 106 ms, sys: 0 ns, total: 106 ms
Wall time: 104 ms


In [180]:
rho1ij

TensorMap with 60 blocks
keys: ['order_nu' 'inversion_sigma' 'spherical_harmonics_l' 'species_center' 'species_neighbor']
           2             1                    0                   1                1
           2             1                    0                   1                6
           2             1                    0                   1                7
        ...
           2            -1                    2                   8                1
           2            -1                    2                   8                6
           2            -1                    2                   8                8

In [181]:
rho1ijp= cg_increment(rho1ip, gji, lcut=3, other_keys_match=["species_contract"], clebsch_gordan=cg, mp=True)

In [182]:
# rho2ijp= cg_increment(rho2ip, gji, lcut=3, other_keys_match=["species_contract"], clebsch_gordan=cg, mp=True)

In [183]:
rho11ijp = cg_combine(rho1i,rho1ijp, lcut=3, other_keys_match=["species_center"], clebsch_gordan=cg)
#cg_combine because we want the usual combination of species_center

In [184]:
elements = np.unique(np.hstack([f.numbers for f in frames]))

In [185]:
rhoMPi = contract_rho_ij(rhoijp, elements)

In [193]:
rhoMP1ij = cg_increment(rhoMPi, gij, lcut=2, other_keys_match=["species_center"], clebsch_gordan=cg)
rhoMP1ijp = relabel_key_contract(rhoMP1ij)

In [194]:
rhoMP11ijp = cg_combine(rho1i,rhoMP1ijp, lcut=3, other_keys_match=["species_center"], clebsch_gordan=cg)

In [195]:
rhoMP11ijp

TensorMap with 96 blocks
keys: ['order_nu' 'inversion_sigma' 'spherical_harmonics_l' 'species_center' 'species_contract']
           5             1                    0                   1                1
           5             1                    0                   1                6
           5             1                    0                   1                7
        ...
           5             1                    3                   8                1
           5             1                    3                   8                6
           5             1                    3                   8                8

In [196]:
rhoMPi_2layers = contract_rho_ij(rhoMP11ijp, elements)

In [197]:
rhoMPi_2layers

TensorMap with 32 blocks
keys: ['order_nu' 'inversion_sigma' 'spherical_harmonics_l' 'species_center']
           5             1                    0                   8
           5             1                    1                   6
           5            -1                    2                   1
        ...
           5            -1                    0                   1
           5            -1                    0                   7
           5             1                    2                   8

In [198]:
rhoMPi_2layers.block(6).samples

Labels([( 0, 1), ( 0, 2), ( 1, 1), ( 1, 2), ( 2, 1), ( 2, 2), ( 3, 1),
        ( 3, 2), ( 4, 1), ( 4, 2), ( 5, 1), ( 5, 2), ( 6, 1), ( 6, 2),
        ( 7, 1), ( 7, 2), ( 8, 1), ( 8, 2), ( 9, 1), ( 9, 2), (10, 3),
        (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (11, 1), (11, 2),
        (11, 3)], dtype=[('structure', '<i4'), ('center', '<i4')])

In [67]:
# selblockidx = np.where(rho1ij.keys['species_neighbor_b'] ==rho1ij.keys['species_center_a'])[0]
# rho1ijp_blocks = []
# rho1ijp_keys = rho1ij.keys[selblockidx]
# for idx in selblockidx: 
#     block = rho1ij.block(int(idx))
#     b = TensorBlock(values=block.values,
#                     samples=block.samples,
#                     components=block.components,
#                     properties= block.properties)
#     rho1ijp_blocks.append(b)
        
# rho1ijp = TensorMap(rho1ijp_keys, rho1ijp_blocks)
#     if k in rho1ijp_key

## OLD CODE 

In [50]:
rhoMPi_keys = list(set([tuple(list(x)[:-1]) for x in rhoijp.keys]))
rhoMPi_blocks = []

for key in rhoMPi_keys:
    contract_blocks=[]
    contract_properties=[]
    contract_samples=rho1i.block(rho1i.blocks_matching(species_center=key[-1])[0]).samples
    for ele in elements:
        block_samples=[]
        blockidx = rhoijp.blocks_matching(species_contract= ele)
        sel_blocks = [rhoijp.block(i) for i in blockidx if key==tuple(list(rhoijp.keys[i])[:-1])]
        print(key,ele, contract_samples)
        if not len(sel_blocks):
            print("skipped")
            continue
        assert len(sel_blocks)==1 #sel_blocks is the corresponding rho11 block with the same key and species_contract = ele
        block = sel_blocks[0]
        filter_idx = list(zip(block.samples['structure'], block.samples['center']))
#             len(block.samples)==len(filter_idx)
        struct, center = np.unique(block.samples['structure']), np.unique(block.samples['center'])
        possible_block_samples = list(product(struct,center))
##             print(block_samples, filter_idx)
        block_values = []
        for sample in contract_samples:
            sample_idx = [idx for idx, tup in enumerate(filter_idx) if tup[0] ==sample[0] and tup[1] == sample[1]]
            if len(sample_idx)==0:
                continue
            block_values.append(block.values[sample_idx].sum(axis=0))
            block_samples.append(sample)
            print(key, ele, block.values[sample_idx].shape, block_values[-1].shape, sample)#sample, block.samples[sample_idx]
        contract_blocks.append(block_values)
        contract_properties.append(block.properties.asarray())
        print(len(contract_samples), len(block_values))
#         contract_samples.append(block_samples)
        
    block_samples=sorted(list(set().union(*contract_samples)))
    if np.asarray(contract_blocks).dtype == 'object' and len(np.asarray(contract_blocks).shape) == 1:# jagged list of samples from different ele of neighbor species
# #         block_samples=sorted(list(set().union(*contract_samples)))
        
        for i in range(len(contract_blocks)):
            N = len(max(contract_blocks, key = lambda x: len(x))) - len(contract_blocks[i])
            print(np.asarray(contract_blocks[i]).shape)
            contract_blocks[i]=np.pad(np.asarray(contract_blocks[i]), ((N,0),(0,0),(0,0)))
            print(contract_blocks[i].shape)
    new_block = TensorBlock(values = np.concatenate(contract_blocks, axis=-1),
                                  samples = contract_samples ,
                                 components = block.components,
                                 properties= Labels(list(rhoijp.property_names), np.asarray(np.vstack(contract_properties),np.int32))
                                 )
    print(key, block_samples, np.asarray(contract_blocks).shape)
    rhoMPi_blocks.append(new_block)
rhoMPi = TensorMap(Labels(['order_nu','inversion_sigma','spherical_harmonics_l','species_center'],np.asarray(rhoMPi_keys, dtype=np.int32)), rhoMPi_blocks)

(3, -1, 0, 8) 1 [(0, 0) (1, 2)]
(3, -1, 0, 8) 1 (2, 1, 512) (1, 512) (0, 0)
(3, -1, 0, 8) 1 (6, 1, 512) (1, 512) (1, 2)
2 2
(3, -1, 0, 8) 6 [(0, 0) (1, 2)]
(3, -1, 0, 8) 6 (2, 1, 512) (1, 512) (1, 2)
2 1
(3, -1, 0, 8) 7 [(0, 0) (1, 2)]
skipped
(3, -1, 0, 8) 8 [(0, 0) (1, 2)]
(3, -1, 0, 8) 8 (1, 1, 512) (1, 512) (0, 0)
(3, -1, 0, 8) 8 (1, 1, 512) (1, 512) (1, 2)
2 2
(2, 1, 512)
(2, 1, 512)
(1, 1, 512)
(2, 1, 512)
(2, 1, 512)
(2, 1, 512)
(3, -1, 0, 8) [0, 1, 2] (3, 2, 1, 512)
(3, 1, 0, 1) 1 [(0, 1) (0, 2) (1, 3) (1, 4) (1, 5) (1, 6) (1, 7) (1, 8) (2, 1) (2, 2)
 (2, 3)]
(3, 1, 0, 1) 1 (2, 1, 1408) (1, 1408) (0, 1)
(3, 1, 0, 1) 1 (2, 1, 1408) (1, 1408) (0, 2)
(3, 1, 0, 1) 1 (6, 1, 1408) (1, 1408) (1, 3)
(3, 1, 0, 1) 1 (6, 1, 1408) (1, 1408) (1, 4)
(3, 1, 0, 1) 1 (5, 1, 1408) (1, 1408) (1, 5)
(3, 1, 0, 1) 1 (5, 1, 1408) (1, 1408) (1, 6)
(3, 1, 0, 1) 1 (5, 1, 1408) (1, 1408) (1, 7)
(3, 1, 0, 1) 1 (3, 1, 1408) (1, 1408) (1, 8)
(3, 1, 0, 1) 1 (3, 1, 1408) (1, 1408) (2, 1)
(3, 1, 0, 1) 1 (3, 1,

/tmp/ipykernel_27464/4169475587.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if np.asarray(contract_blocks).dtype == 'object' and len(np.asarray(contract_blocks).shape) == 1:# jagged list of samples from different ele of neighbor species


(3, -1, 2, 1) [0, 1, 2, 3, 4, 5, 6, 7, 8] (4, 11, 5, 2176)
(3, -1, 0, 1) 1 [(0, 1) (0, 2) (1, 3) (1, 4) (1, 5) (1, 6) (1, 7) (1, 8) (2, 1) (2, 2)
 (2, 3)]
(3, -1, 0, 1) 1 (2, 1, 512) (1, 512) (0, 1)
(3, -1, 0, 1) 1 (2, 1, 512) (1, 512) (0, 2)
(3, -1, 0, 1) 1 (6, 1, 512) (1, 512) (1, 3)
(3, -1, 0, 1) 1 (6, 1, 512) (1, 512) (1, 4)
(3, -1, 0, 1) 1 (5, 1, 512) (1, 512) (1, 5)
(3, -1, 0, 1) 1 (5, 1, 512) (1, 512) (1, 6)
(3, -1, 0, 1) 1 (5, 1, 512) (1, 512) (1, 7)
(3, -1, 0, 1) 1 (3, 1, 512) (1, 512) (1, 8)
(3, -1, 0, 1) 1 (3, 1, 512) (1, 512) (2, 1)
(3, -1, 0, 1) 1 (3, 1, 512) (1, 512) (2, 2)
(3, -1, 0, 1) 1 (3, 1, 512) (1, 512) (2, 3)
11 11
(3, -1, 0, 1) 6 [(0, 1) (0, 2) (1, 3) (1, 4) (1, 5) (1, 6) (1, 7) (1, 8) (2, 1) (2, 2)
 (2, 3)]
(3, -1, 0, 1) 6 (2, 1, 512) (1, 512) (1, 3)
(3, -1, 0, 1) 6 (2, 1, 512) (1, 512) (1, 4)
(3, -1, 0, 1) 6 (2, 1, 512) (1, 512) (1, 5)
(3, -1, 0, 1) 6 (2, 1, 512) (1, 512) (1, 6)
(3, -1, 0, 1) 6 (2, 1, 512) (1, 512) (1, 7)
(3, -1, 0, 1) 6 (2, 1, 512) (1, 512) (1

In [42]:
rhoMPi_keys = list(set([tuple(list(x)[:-1]) for x in rhoijp.keys]))
rhoMPi_blocks = []

for key in rhoMPi_keys:
    contract_blocks=[]
    contract_properties=[]
    contract_samples=[]
    for ele in elements:
        block_samples=[]
        blockidx = rhoijp.blocks_matching(species_contract= ele)
        sel_blocks = [rhoijp.block(i) for i in blockidx if key==tuple(list(rhoijp.keys[i])[:-1])]
        print(key,ele)
        if not len(sel_blocks):
            continue
        assert len(sel_blocks)==1 #sel_blocks is the corresponding rho11 block with the same key and species_contract = ele
        block = sel_blocks[0]
        filter_idx = list(zip(block.samples['structure'], block.samples['center']))
#             len(block.samples)==len(filter_idx)
        struct, center = np.unique(block.samples['structure']), np.unique(block.samples['center'])
        possible_block_samples = list(product(struct,center))
##             print(block_samples, filter_idx)
        block_values = []
        for sample in possible_block_samples:
            sample_idx = [idx for idx, tup in enumerate(filter_idx) if tup[0] ==sample[0] and tup[1] == sample[1]]
            if len(sample_idx)==0:
                continue
            block_values.append(block.values[sample_idx].sum(axis=0))
            block_samples.append(sample)
#             print(key, ele, sample, block.samples[sample_idx],block.values[sample_idx].shape)
        contract_blocks.append(block_values)
        contract_properties.append(block.properties.asarray())
        contract_samples.append(block_samples)
        
    block_samples=sorted(list(set().union(*contract_samples)))
    if not len(block_samples) or (np.asarray(contract_samples).dtype == 'object' and len(np.asarray(contract_samples).shape) == 1):# jagged list of samples from different ele of neighbor species
#         block_samples=sorted(list(set().union(*contract_samples)))
        for i in range(len(contract_samples)):
            obj=sorted([block_samples.index(i) for i in list(set(block_samples).difference(contract_samples[i]))])#, reverse=True)
            if len(obj)!=0:
                print(obj, len(contract_samples),i)
                contract_blocks[i]=np.insert(contract_blocks[i], obj, np.zeros(contract_blocks[i][0].shape), axis=0)

    new_block = TensorBlock(values = np.concatenate(contract_blocks, axis=-1),
                                  samples = Labels(["structure", "center"], np.asarray(block_samples, dtype=np.int32)) ,
                                 components = block.components,
                                 properties= Labels(list(rhoijp.property_names), np.asarray(np.vstack(contract_properties),np.int32))
                                 )
    print(key, block_samples, np.asarray(contract_blocks).shape)
    rhoMPi_blocks.append(new_block)

rhoMPi = TensorMap(Labels(['order_nu','inversion_sigma','spherical_harmonics_l','species_center'],np.asarray(rhoMPi_keys, dtype=np.int32)), rhoMPi_blocks)

(3, 1, 1, 7) 1
(3, 1, 1, 7) 6
(3, 1, 1, 7) 7
(3, 1, 1, 7) 8
(3, 1, 1, 7) [(2, 0)] (2, 1, 3, 2688)
(3, 1, 2, 7) 1
(3, 1, 2, 7) 6
(3, 1, 2, 7) 7
(3, 1, 2, 7) 8
(3, 1, 2, 7) [(2, 0)] (2, 1, 5, 2816)
(3, -1, 1, 6) 1
(3, -1, 1, 6) 6
(3, -1, 1, 6) 7
(3, -1, 1, 6) 8
(3, -1, 1, 6) [(1, 0), (1, 1)] (3, 2, 3, 1920)
(3, 1, 0, 6) 1
(3, 1, 0, 6) 6
(3, 1, 0, 6) 7
(3, 1, 0, 6) 8
(3, 1, 0, 6) [(1, 0), (1, 1)] (3, 2, 1, 1408)
(3, 1, 3, 7) 1
(3, 1, 3, 7) 6
(3, 1, 3, 7) 7
(3, 1, 3, 7) 8
(3, 1, 3, 7) [(2, 0)] (2, 1, 7, 1920)
(3, -1, 2, 6) 1
(3, -1, 2, 6) 6
(3, -1, 2, 6) 7
(3, -1, 2, 6) 8
(3, -1, 2, 6) [(1, 0), (1, 1)] (3, 2, 5, 2176)
(3, 1, 3, 1) 1
(3, 1, 3, 1) 6
(3, 1, 3, 1) 7
(3, 1, 3, 1) 8
[0, 1, 8, 9, 10] 4 1


/Users/jigyasa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


IndexError: index 12 is out of bounds for axis 0 with size 11